In [ ]:
import pandas as pd
import requests
import json

In [ ]:
#referred: https://github.com/AdamGetbags/secAPI/blob/main/secFilingScraper.py
# https://www.youtube.com/watch?v=Wr1NoM3JkTo

In [ ]:
web_site='https://www.sec.gov/files/company_tickers.json'
df=pd.read_json(web_site)

In [ ]:
#df.T

In [ ]:
df=df.reindex(['ticker','title','cik_str'])

In [ ]:
df_new=df.T

In [ ]:
df_new

In [ ]:
#10 digit CIK for data referencing
df_new['CIK']=df_new['cik_str'].apply(lambda x:str(x).zfill(10))

In [ ]:
df_new

In [ ]:
df_new.set_index('ticker',inplace=True)

In [ ]:
st_name ='MSFT'
cik_name=df_new.loc[st_name]['CIK']
file='CIK'+cik_name+'.json'
print(f"""
      stock_name: {st_name}
      cik_name  : {cik_name}
      file_name : {file}
      """)


In [ ]:
#path to company facts
#file='CIK0001753391.json'
json_file=f'/home/thakur/Downloads/Aacompnayfacts/{file}'
print(json_file)

# JSON FILE

In [ ]:
with open(json_file) as jfile:
    jf = json.load(jfile)



In [ ]:
print(jf.keys())

In [ ]:
# keys of ['facts']['us-gaap']
print(pd.DataFrame(jf['facts']['us-gaap'].keys(),columns=['Variable']).to_string())

In [ ]:
parameter='Revenues'
rev=jf['facts']['us-gaap'][parameter]['units']['USD']
rev

In [ ]:
#dataframe of revenue
rev_df=pd.DataFrame(rev)
rev_df

In [ ]:
df_new.loc['AAPL']['cik_str']

In [ ]:
# Tickers scrappter
# create request header
headers = {'User-Agent': "bhattathakur2015@gmail.com"}

In [ ]:
# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

In [ ]:
companyTickers

In [ ]:
# review response / keys
#print(companyTickers.json().keys())

In [ ]:
# format response to dictionary and get first key/value
firstEntry = companyTickers.json()['0']

# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

In [ ]:
directCik

In [ ]:
# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

In [ ]:
companyData

In [ ]:
# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

In [ ]:
# review data
print(companyData[:1])
cik = companyData[0:1].cik_str[0]

In [ ]:
# get company specific filing metadata
filingMetadata = requests.get(
    f'https://data.sec.gov/submissions/CIK{cik}.json',
    headers=headers
    )


In [ ]:
# review json 
print(filingMetadata.json().keys())

In [ ]:
# get company concept data
companyConcept = requests.get(
    (
    f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}'
     f'/us-gaap/Assets.json'
    ),
    headers=headers
    )

In [ ]:
# get all filings data 
assetsData = pd.DataFrame.from_dict((
               companyConcept.json()['units']['USD']))

In [ ]:
# get all filings data 
assetsData = pd.DataFrame.from_dict((
               companyConcept.json()['units']['USD']))

In [ ]:
# get assets from 10Q forms and reset index
assets10Q = assetsData[assetsData.form == '10-Q']
assets10Q = assets10Q.reset_index(drop=True)

In [ ]:
assets10Q.plot(x='end',y='val')

In [ ]:
assets10Q

In [ ]:
assetsData

In [ ]:
assets10Q = assetsData[assetsData.form == '10-Q']
assets10Q

In [ ]:
assets10Q[-3:].plot(x='end',y='val')

In [ ]:
assets10Q.frame

In [ ]:
companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )

In [ ]:
companyFacts.json()['facts']['us-gaap']['Revenues']

In [ ]:
companyFacts.json()['facts']['us-gaap'].keys()

In [ ]:
# dictionary to dataframe
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
             )

In [ ]:
allForms.columns

In [ ]:
allForms[['accessionNumber', 'reportDate', 'form']].iloc[11]

In [ ]:
# dictionary to dataframe
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
             ).iloc[11]

In [ ]:
allForms